# Pandas Tips & Tricks

This notebook presents various tricks to manipulate your data, which are typically non-obvious to a novice in Pandas and data science.

In [ ]:
import numpy as np
import pandas as pd

## Reading Data

### Combining Multiple CSV Files

This shows how to create a single dataframe from multiple files that share the same structure (columns).

In [ ]:
import glob

files = '../data/commits*.tsv'
df = pd.concat([pd.read_csv(x, sep='\t') for x in glob.glob(files)], 
               ignore_index=True)

!wc -l {files}
print('♯', len(df))

In [ ]:
len(df.Author.unique()), len(pd.unique(df.Author))

## Inspecting Dataframes

Looking at the contents and metadata of your dataframes is quite important, to better understand the data they represent and then successfully transform it into the results you need.

In [ ]:
# Data dimensions (rows, cols)
df.shape

In [ ]:
# Data types
df.dtypes

If you look at a sample, it is often useful to transpose the data, especially when you have many columns.

In [ ]:
df.head(3).transpose()

And then there is `describe` with some core statistics about the dataframe…

In [ ]:
df.describe()

… and `info` with more technical information.

In [ ]:
df.info()

## Writing Results
### Writing Spreadsheet Files
*TODO*

## Filtering Rows

You can use [loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) in combination with a `bool` array to select a subset of rows. That array is conveniently created by applying conditions to columns.

The first example uses regex matching…

In [ ]:
df.loc[df.Message.str.match('altair', case=False)]

Another option is using simple comparison operators, e.g. `!=` like here…

In [ ]:
df.loc[df.Author != 'jhermann']

Note that the condition creates a `bool` array, that then is taken by `loc[…]` to select the matching rows.

In [ ]:
list(df.Author.iloc[-5:] != 'jhermann')

## Manipulating Columns

### Adding or Replacing Columns
Changing the values of a column or adding a whole new one can be done by actual assignment or by calling `assign`.

In [ ]:
morecols = df.assign(Words=df.Message.str.split().apply(len))
print('Column ♯:', len(df.columns), 'vs.', len(morecols.columns))
morecols.head(2)

Using assigment is inplace and changes the dataframe.

In [ ]:
morecols['Zero'] = 0
morecols.head(2)

### Renaming Columns
This one's easy, just call [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html). Columns can be specified in various formats, like a mapping from old to new. Renaming can also be done inplace, the default is to copy.

In [ ]:
morecols.rename(columns=dict(Message='Text')).head(1)

To rename all columns, just `zip` the existing names with the new ones.

In [ ]:
morecols.rename(columns=dict(zip(morecols.columns, range(len(morecols.columns))))).head(1)

To rename according to some logic, like a regex substitution or similar, provide a mapper function.

In [ ]:
morecols.rename(mapper=str.upper, axis=1).head(1)

### Deleting Columns
*TODO*

### Selecting Columns

In [ ]:
df[['Date', 'Message']].head(1)

## String Manipulation

The new `Day` column is just the first word out of the `Date` column. By splitting with `expand=True` two columns are created (instead of one column with tuples), so we can select the first column only and add this to the dataframe.

In [ ]:
df = df.assign(Day=df.Date.str.split(n=1, expand=True)[0])
df.head(1)

## Counting

To visualize data in bar or other magnitude charts, you have to count subsets of your raw data.

In [ ]:
commits_per_day = df.Day.value_counts().to_frame().sort_index()
_ = commits_per_day.plot.barh(legend=False, figsize=(5, 2))

## Aggregation

Grouping values by one or more columns and then applying an operation to fold those values into a single scalar.

In [ ]:
letters = list("Pandas")
codes = pd.DataFrame(dict(Letter=letters, Code=list(map(ord, letters))))
codes = codes.groupby('Letter').aggregate(np.sum)
codes.head(1)

Using `reset_index` moves the grouping column(s) from the index to ordinary columns.

In [ ]:
codes = codes.reset_index()
codes